<a href="https://colab.research.google.com/github/mohammad-gh009/AutoGPT/blob/master/main_paper_maker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install accelerate
!pip install -U bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 6.4 MB/s eta 0:00:00


In [ ]:
from transformers import BitsAndBytesConfig

nf4_config = BitsAndBytesConfig(
    load_in_bbit=True,
    bnb_4bit_quant_type="nf4",
)



In [ ]:
import torch

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("h2oai/h2ogpt-gm-7b-mistral-chat-sft-dpo-rag-v1")
model = AutoModelForCausalLM.from_pretrained("h2oai/h2ogpt-gm-7b-mistral-chat-sft-dpo-rag-v1", device_map="auto", load_in_4bit=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
messages = [
    {
        "role": "system",
        "content": "You are an expert in extracting numbers from a text.",
    },
    {"role": "user", "content": """JUST give me ONE python list of citation indexes in numbers WITHOUT any other words from the following text:4.2. Model Application
Not all the models are appropriate for all problems. Selecting a model depends on fac-
tors such as the problem’s nature, the data’s size and quality, interpretability requirements,
computational resources, and so on. A problem might be of the classiﬁcation, regression, or
clustering type. Moreover, in a different taxonomy, either of the following types of models
might be appropriate for a problem:
•
Supervised models: the algorithms learn from labeled data, where the input data are
paired with corresponding target or output values. The goal is to predict these target
values for new, unseen data. A few examples of supervised models are linear [133]
and logistic regressions [134], decision trees [129], random forests [135], support vector
machines [136], Convolutional Neural Networks [137], and Recurrent Neural Net-
works [138]. Some examples, such as linear regression, could be used for classiﬁcation
(ﬁnding the category of the data) and regression (ﬁnding the numerical value of the
data), and some are speciﬁc to classiﬁcation or regression.
•
Unsupervised models: the algorithms work with unlabeled data, seeking to discover
patterns, structures, or relationships within the data without explicit guidance on
what to look for. They are also known as clustering algorithms. A few examples of
unsupervised models are K-Means [139], Hierarchical Clustering [140], and Generative
Adversarial Networks [141].
•
Reinforcement learning: an agent learns to make decisions by interacting with an
environment. It receives feedback in the form of rewards or penalties based on its
actions to maximize cumulative rewards over time. The most well-known example
of this category is Q-Learning [142], which is used in tasks such as tic-tac-toe game
playing and simple robot control.
Moreover, a speciﬁc model is not guaranteed to be better than others. Choosing a
model depends on various parameters, as mentioned before. However, algorithm selection
is often an iterative process that needs reﬁning as more insights from experiments and data
analysis are obtained."""},
 ]

In [ ]:
def generator(messages):
  tokenized_chat = tokenizer.apply_chat_template(messages , tokenize=True, add_generation_prompt=True, return_tensors="pt")
  outputs = model.generate(tokenized_chat, max_new_tokens= 500)
  text = tokenizer.decode(outputs[0]).split("<|answer|>")[1]
  return text

In [ ]:
x = generator(messages)
x

' Here is a Python list of citation indexes in numbers without any other words:\n\n[133, 134, 129, 135, 136, 137, 138, 139, 140, 141, 142]</s>'

In [ ]:
len(tokenizer.tokenize(x))

In [ ]:
generator(messages)

In [ ]:
tokenized_chat = tokenizer.apply_chat_template(messages , tokenize=True, add_generation_prompt=True, return_tensors="pt")
outputs = model.generate(tokenized_chat, max_new_tokens= 1500)
text = tokenizer.decode(outputs[0]).split("<|answer|>")[1]

In [ ]:
messages = [
    {
        "role": "system",
        "content": "You are an expert medical journal editor.",
    },
    {"role": "user", "content":"edit the following  paragraph :"+text },
 ]

In [ ]:
tokenized_chat = tokenizer.apply_chat_template(messages , tokenize=True, add_generation_prompt=True, return_tensors="pt")

In [ ]:
outputs = model.generate(tokenized_chat, max_new_tokens= 1000)

In [ ]:
len(outputs[0])

608

In [ ]:
print(tokenizer.decode(outputs[0]))

<|system|>You are an expert medical journal editor.</s> <|prompt|>edit the following  paragraph : The differential diagnosis of diabetes insipidus involves recognizing the differences between central or nephrogenic diabetes insipidus and primary polydipsia. It's crucial to differentiate these conditions because the treatments for each one may differ significantly. Reliable differentiation can be challenging, particularly when dealing with patients with primary polydipsia or partial forms of diabetes insipidus.

Recent diagnostic algorithms rely on measuring copeptin levels after osmotic stimulation by hypertonic saline infusion or after nonosmotic stimulation by arginine. These methods have a higher diagnostic accuracy compared to the water deprivation test. In cases of central diabetes insipidus, nephrogenic diabetes insipidus, or primary polydipsia, treatment typically involves correcting existing water deficits. However, the specific approach to treatment will depend on the underlyi

In [ ]:
text = tokenizer.decode(outputs[0]).split("<|answer|>")[1]

In [ ]:
text

" to ADH secretion, leading to excessive thirst and fluid loss. This condition is usually caused by a disorder of the renal tubules or the collecting ducts. Treatment typically involves replacing the missing ADH, either through intranasal administration of synthetic ADH or by intravenous infusion of recombinant human ADH.\n\nPrimary polydipsia is a rare condition characterized by excessive thirst without any identifiable cause. It can be caused by various factors, including genetic mutations, autoimmune diseases, or metabolic disorders. Treatment typically involves managing the patient's thirst and hydration needs, as well as monitoring their electrolyte balance and blood pressure.\n\nIn summary, the differential diagnosis of diabetes insipidus involves recognizing the differences between central or nephrogenic diabetes insipidus and primary polydipsia. Treatment approaches for each condition will vary depending on the underlying cause, but both involve addressing the patient's thirst 